In [13]:
import pandas as pd
import numpy as np
import yfinance as yf
import os
from datetime import datetime
from datetime import timedelta
import matplotlib as mt

## Create Function to Import Data from Yahoo Finance

In [2]:
def get_yfinance_data(ticker_name, period):
    ticker = ticker_name
    period = period
    
    data = yf.Ticker(ticker)
    historical_close = pd.DataFrame(data.history(period = period))["Close"]
    return(historical_close)

In [3]:
yahoo_data_list = ["SPY", "DJI"]

In [4]:
for ticker in yahoo_data_list:
    data = pd.DataFrame(get_yfinance_data(ticker, "max"))

## Read CSV Files in the Folder

In [5]:
# Adding working directory
directory = os.listdir("C:/Users/peter/Desktop/python-recession-model")

In [6]:
csv_file_list = []
csv_name = []
for file in directory:
    if file.endswith(".csv"):
        data_name = file.split(".")[0]
        data = pd.read_csv(file)
        csv_name.append(data_name)
        csv_file_list.append(data)

In [7]:
csv_name1 = pd.DataFrame(csv_name).apply(lambda x: x.str.lower().str.replace('(', '').str.replace(')', '').str.replace('-', '').str.replace(',', ' ').str.replace(' ', '_').str.replace('__', '_'))

C:\Users\peter\AppData\Local\Temp\ipykernel_22432\4020979771.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  csv_name1 = pd.DataFrame(csv_name).apply(lambda x: x.str.lower().str.replace('(', '').str.replace(')', '').str.replace('-', '').str.replace(',', ' ').str.replace(' ', '_').str.replace('__', '_'))


In [8]:
csv_name1

,0
0,capacity_utilization_total_index
1,consumer_price_index_for_all_urban_consumers_a...
2,cpi
3,crude_oil_prices_west_texas_intermediate_wti_c...
4,dji
5,economic_policy_uncertainty_index_for_united_s...
6,federal_funds
7,federal_funds_effective_rate_daily
8,fed_model_data
9,industrial_production_total_index


In [9]:
table_dict = {}

for i in range(30):
        table_name = csv_name1.at[i, 0]
        data = pd.DataFrame(csv_file_list[i])
        table_dict[table_name] = data

In [10]:
# Format the date column name
for keys, values in table_dict.items():
    table_dict[keys] = values.rename(str.lower, axis = "columns")

In [11]:
table_count = 0
for keys, values in table_dict.items():
    number_row = len(values)
    top_two = values.head(2)
    bottom_two = values.tail(2)
    table_count = table_count + 1
    print("Table Number: " + str(table_count))
    print("Table Name: " + keys)
    print("Number of Row: "  + str(number_row))
    print(top_two)
    print(bottom_two)
    print("____________________________________________________________________________________________")

Table Number: 1
Table Name: capacity_utilization_total_index
Number of Row: 681
         date      tcu
0  1967-01-01  89.3902
1  1967-02-01  87.9750
           date      tcu
679  2023-08-01  79.5385
680  2023-09-01  79.6786
____________________________________________________________________________________________
Table Number: 2
Table Name: consumer_price_index_for_all_urban_consumers_all_items_in_u
Number of Row: 921
         date  cpiaucsl
0  1947-01-01     21.48
1  1947-02-01     21.62
           date  cpiaucsl
919  2023-08-01   306.269
920  2023-09-01   307.481
____________________________________________________________________________________________
Table Number: 3
Table Name: cpi
Number of Row: 914
         date  cpiaucsl
0  1947-01-01     21.48
1  1947-02-01     21.62
           date  cpiaucsl
912  2023-01-01   300.536
913  2023-02-01   301.648
____________________________________________________________________________________________
Table Number: 4
Table Name: crude_oil_p

# Format Date Object

In [12]:
def guess_date(string):
    for date_format in ["%Y-%m-%d", "%m/%d/%Y", "%d-%b-%y"]:
        try:
            return datetime.strptime(string, date_format).date()
        except ValueError:
            continue
    raise ValueError(string)

In [13]:
for keys, values in table_dict.items():
    date_string = values["date"]
    date_format = ["%Y-%m-%d", "%m/%d/%Y", "%d-%b-%Y"]
    values["date"] = date_string.apply(lambda x: guess_date(x)).apply(lambda x: x.replace(year = x.year - 100) if x.year > 2030 else x )

In [14]:
# Edge Case Fed Model Date Handling
delta = timedelta(days = 1)
table_dict["fed_model_data"]["date"] = table_dict["fed_model_data"]["date"].apply(lambda x: x + delta)

## Basic Information of each dataset

In [15]:
# Function to provide basic information about a list of dataset
def data_basic_infor(table_dict):
    table_count = 0
    for keys, values in table_dict.items():
        number_row = len(values)
        start_date = min(values["date"])
        end_date = max(values["date"])
        table_count = table_count + 1
        column_name = values.columns
        frequency = values["date"][1] - values["date"][0]
        print("Table Number: " + str(table_count))
        print("Table Name: " + keys)
        print("Column Names: " + str(column_name[0]) + ", " + str(column_name[1]))
        print("Number of Row: "  + str(number_row))
        print("Frequency: " + str(frequency))
        print("Start Date: " + str(start_date))
        print("End Date: " + str(end_date))
        print("____________________________________________________________________________________________")

In [16]:
# Overview of basic information about each dataset
data_basic_infor(table_dict)

Table Number: 1
Table Name: capacity_utilization_total_index
Column Names: date, tcu
Number of Row: 681
Frequency: 31 days, 0:00:00
Start Date: 1967-01-01
End Date: 2023-09-01
____________________________________________________________________________________________
Table Number: 2
Table Name: consumer_price_index_for_all_urban_consumers_all_items_in_u
Column Names: date, cpiaucsl
Number of Row: 921
Frequency: 31 days, 0:00:00
Start Date: 1947-01-01
End Date: 2023-09-01
____________________________________________________________________________________________
Table Number: 3
Table Name: cpi
Column Names: date, cpiaucsl
Number of Row: 914
Frequency: 31 days, 0:00:00
Start Date: 1947-01-01
End Date: 2023-02-01
____________________________________________________________________________________________
Table Number: 4
Table Name: crude_oil_prices_west_texas_intermediate_wti_cushing_oklahoma
Column Names: date, dcoilwtico
Number of Row: 9863
Frequency: 1 day, 0:00:00
Start Date: 1986-0

## Filter the Data and Create Updated List of Dataset

In [17]:
# Filter the list to delete based on the following requirements:
# 1. Has start date before 1975
# 2. Has end date after 2023
# 3. The update frequency is less than a month

list_to_delete = []
delta = timedelta(days = 32)
for keys, values in table_dict.items():
    frequency = values["date"][1] - values["date"][0]
    if min(values["date"]).year > 1975:
        list_to_delete.append(keys)
    elif frequency > delta:
        list_to_delete.append(keys)
    elif max(values["date"]).year < 2023:
        list_to_delete.append(keys)
        
        
print("The delete list has " + str(len(list_to_delete)) + " and has following items: ")
print("")
for table_name in list_to_delete:
    print(table_name)

The delete list has 10 and has following items: 

crude_oil_prices_west_texas_intermediate_wti_cushing_oklahoma
dji
economic_policy_uncertainty_index_for_united_states
inflation_consumer_prices_for_the_united_states
leading_index_for_the_united_states
t10y2y_daily
t10y3m_daily
university_of_michigan_inflation_expectation
velocity_of_m2_money_stock
vix


In [18]:
updated_table_dict = table_dict

for i in list_to_delete:
    del updated_table_dict[i]


In [19]:
data_basic_infor(updated_table_dict)

Table Number: 1
Table Name: capacity_utilization_total_index
Column Names: date, tcu
Number of Row: 681
Frequency: 31 days, 0:00:00
Start Date: 1967-01-01
End Date: 2023-09-01
____________________________________________________________________________________________
Table Number: 2
Table Name: consumer_price_index_for_all_urban_consumers_all_items_in_u
Column Names: date, cpiaucsl
Number of Row: 921
Frequency: 31 days, 0:00:00
Start Date: 1947-01-01
End Date: 2023-09-01
____________________________________________________________________________________________
Table Number: 3
Table Name: cpi
Column Names: date, cpiaucsl
Number of Row: 914
Frequency: 31 days, 0:00:00
Start Date: 1947-01-01
End Date: 2023-02-01
____________________________________________________________________________________________
Table Number: 4
Table Name: federal_funds
Column Names: date, fedfunds
Number of Row: 828
Frequency: 31 days, 0:00:00
Start Date: 1954-07-01
End Date: 2023-06-01
_______________________

# Merge for final table

In [8]:
final_data = updated_table_dict["capacity_utilization_total_index"]

for keys, values in updated_table_dict.items():
    print(keys)
    final_data = final_data.merge(values, how = "left", on = "date")

NameError: name 'updated_table_dict' is not defined

In [21]:
updated_final_data = final_data.drop(labels = ["icsa", "dgs10"], axis = 1)

In [22]:
updated_final_data["spread"] != None

0      True
1      True
2      True
3      True
4      True
       ... 
676    True
677    True
678    True
679    True
680    True
Name: spread, Length: 681, dtype: bool

In [23]:
updated_final_data = updated_final_data[-(updated_final_data["spread"].isna())].dropna().reset_index()

In [24]:
updated_final_data.isna().sum()

index                                             0
date                                              0
tcu_x                                             0
tcu_y                                             0
cpiaucsl_x                                        0
cpiaucsl_y                                        0
fedfunds                                          0
dff                                               0
10 year treasury yield                            0
3 month treasury yield                            0
3 month treasury yield (bond equivalent basis)    0
spread                                            0
rec_prob                                          0
nber_rec                                          0
indpro                                            0
civpart                                           0
m1sl                                              0
m2sl                                              0
mabmm301usm189s_x                                 0
mabmm301usm1

In [7]:
updated_final_data

NameError: name 'updated_final_data' is not defined

In [26]:
updated_final_data["12m_rec"] = 0

for i in updated_final_data.index:
    
    if i > 648:
        break
    
    rec_count = 0
    for j in range(12):
        rec_count = rec_count + updated_final_data.loc[i + j, "usrec"]
    
    if rec_count != 0:
        rec_count = 1
    
    print(rec_count)
    updated_final_data.loc[i , "12m_rec"] = rec_count

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0


In [27]:
list_to_drop = ["usrec", "10 year treasury yield", "3 month treasury yield", "3 month treasury yield (bond equivalent basis)"]

updated_final_data.drop(labels = list_to_drop, axis = 1)

,index,date,tcu_x,tcu_y,cpiaucsl_x,cpiaucsl_y,fedfunds,dff,spread,rec_prob,...,civpart,m1sl,m2sl,mabmm301usm189s_x,mabmm301usm189s_y,bogmbase,value,unrate,umcsent,12m_rec
0,0,1967-01-01,89.3902,89.3902,32.900,32.900,4.94,5.00,-0.25,26.92%,...,59.5,171.9,481.6,4.816000e+11,4.816000e+11,63800.0,84.45,3.9,.,0
1,1,1967-02-01,87.9750,87.9750,33.000,33.000,5.00,4.00,-0.26,31.88%,...,59.3,173.0,485.1,4.851000e+11,4.851000e+11,63300.0,87.36,3.8,94.1,0
2,2,1967-03-01,87.0835,87.0835,33.000,33.000,4.53,3.75,-0.05,28.40%,...,59.1,174.8,489.7,4.897000e+11,4.897000e+11,63200.0,89.42,3.8,.,0
3,3,1967-04-01,87.5089,87.5089,33.100,33.100,4.05,4.50,0.17,26.25%,...,59.4,174.2,492.1,4.921000e+11,4.921000e+11,63300.0,90.96,3.8,.,0
4,4,1967-05-01,86.3566,86.3566,33.100,33.100,3.94,4.00,0.66,29.46%,...,59.3,175.7,497.2,4.972000e+11,4.972000e+11,63400.0,92.59,3.8,95.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,669,2022-10-01,80.6377,80.6377,297.987,297.987,3.08,3.08,0.32,8.46%,...,62.2,20098.1,21432.1,2.143130e+13,2.143270e+13,5340200.0,3726.05,3.7,59.9,0
656,670,2022-11-01,80.2626,80.2626,298.598,298.598,3.78,3.08,0.17,6.66%,...,62.2,19964.4,21398.7,2.139690e+13,2.139870e+13,5419400.0,3917.49,3.6,56.8,0
657,671,2022-12-01,78.9195,78.9195,298.990,298.990,4.10,3.83,-0.36,6.83%,...,62.3,19821.0,21358.1,2.135770e+13,2.135760e+13,5406000.0,3912.38,3.5,59.7,0
658,672,2023-01-01,79.6374,79.6374,300.536,300.536,4.33,4.33,-0.74,7.70%,...,62.4,19560.1,21212.6,2.121200e+13,2.122170e+13,5329000.0,3960.66,3.4,64.9,0


In [37]:
def get_monthly_change_rate(columns):
    monthly_change = [None]
    
    target_index = (columns.index + 1)
    target_index = target_index.drop([len(target_index)])
    
    for row in target_index:
        change = (columns[row]/columns[row - 1]) - 1
        monthly_change.append(change)
        
    return(monthly_change)
            

In [38]:
spread_change = get_monthly_change_rate(updated_final_data["spread"])

C:\Users\peter\AppData\Local\Temp\ipykernel_22432\115550959.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  change = (columns[row]/columns[row - 1]) - 1


In [39]:
spread_change

[None,
 0.040000000000000036,
 -0.8076923076923077,
 -4.4,
 2.8823529411764706,
 0.7727272727272725,
 0.19658119658119655,
 -0.3928571428571428,
 0.05882352941176472,
 -0.1444444444444445,
 0.038961038961039085,
 0.125,
 -0.33333333333333337,
 -0.33333333333333326,
 0.07499999999999996,
 -0.7441860465116279,
 -0.5454545454545454,
 -0.20000000000000007,
 0.0,
 3.75,
 -0.3157894736842105,
 -0.3846153846153846,
 0.25,
 -2.0999999999999996,
 1.5454545454545459,
 -0.6071428571428572,
 -1.9090909090909092,
 -2.2,
 -1.8333333333333335,
 -1.7000000000000002,
 6.285714285714286,
 0.0,
 -0.6862745098039216,
 -0.1875,
 1.6153846153846154,
 0.2941176470588234,
 -0.20454545454545459,
 -0.6571428571428571,
 -2.916666666666667,
 1.956521739130435,
 0.24999999999999978,
 0.11764705882352944,
 -0.14736842105263148,
 0.1358024691358024,
 0.1847826086956521,
 0.14678899082568808,
 0.1279999999999999,
 -0.014184397163120588,
 0.2158273381294964,
 0.3727810650887573,
 -0.0344827586206895,
 -0.1607142857142

In [41]:
updated_final_data["spread_change"] =  spread_change

In [1]:
updated_final_data

NameError: name 'updated_final_data' is not defined

In [ ]:
updated_final_data.to_excel("final_data.xlsx")

In [48]:
def count_momentum(change_list):
    momentum_list = []
    
    for index in change_list.index:
        
        if change_list[index] == None: 
            momentum_list.append(None)
        
        if change_list[index] >= 0:
            momentum = 0
            momentum_index = index
            
            while change_list[momentum_index] >= 0:
                momentum =+ 1
                momentum_index =+ 1
                
            momentum_list.append(momentum)
            
        if change_list[index] < 0:
            momentum = 0
            momentum_index = index
            
            while change_list[momentum_index] < 0:
                momentum =+ 1
                momentum_index =+ 1
                
            momentum_list.append(momentum)
    
    return momentum_list

In [49]:
count_momentum(updated_final_data["spread_change"])

KeyboardInterrupt: 

In [ ]:
test = 

In [60]:
test = pd.Series([None,       
 0.040000000000000036,
 -0.8076923076923077,
 -4.4,
 2.8823529411764706,
 0.7727272727272725,
 0.19658119658119655,
 -0.3928571428571428,
 0.05882352941176472,
 -0.1444444444444445,
 0.038961038961039085,
 0.125,
 -0.33333333333333337,
 -0.33333333333333326,
 0.07499999999999996,
 -0.7441860465116279,
 -0.5454545454545454,
 -0.20000000000000007,
 0.0,
 3.75,
 -0.3157894736842105,
 -0.3846153846153846,
 0.25,
 -2.0999999999999996,
 1.5454545454545459,
 -0.6071428571428572,
 -1.9090909090909092,
 -2.2,
 -1.8333333333333335,
 -1.7000000000000002,
 6.285714285714286,
 0.0,
 -0.6862745098039216,
 -0.1875,
 1.6153846153846154,
 0.2941176470588234,
 -0.20454545454545459,
 -0.6571428571428571,
 -2.916666666666667,
 1.956521739130435,
 0.24999999999999978,
 0.11764705882352944,
 -0.14736842105263148,
 0.1358024691358024,
 0.1847826086956521,
 0.14678899082568808,
 0.1279999999999999,
 -0.014184397163120588,
 0.2158273381294964,
 0.3727810650887573,
 -0.0344827586206895,
 -0.1607142857142858,
 0.1436170212765957,
 -0.2325581395348837,
 -0.2848484848484848,
 0.2796610169491527,
 -0.11920529801324498,
 0.022556390977443552,
 0.09558823529411753,
 0.2214765100671141,
 0.36813186813186816,
 -0.09638554216867479,
 0.06222222222222218,
 -0.01673640167364021,
 -0.10212765957446812,
 -0.03791469194312802,
 0.029556650246305383,
 -0.15311004784688986,
 -0.0847457627118644,
 -0.154320987654321,
 -0.16058394160583955,
 -0.21739130434782605,
 -0.022222222222222254,
 -0.5,
 -0.5,
 0.36363636363636354,
 -2.7333333333333334,
 1.2307692307692304,
 0.3706896551724139,
 -0.05660377358490576,
 -0.5533333333333332,
 1.044776119402985,
 -0.2992700729927008,
 0.09375,
 -0.6285714285714286,
 1.641025641025641,
 0.08737864077669921,
 -0.1607142857142858,
 -0.32978723404255317,
 -1.0158730158730158,
 -126.0,
 -0.76,
 -1.6333333333333333,
 -1.2105263157894737,
 -2.25,
 20.0,
 0.6476190476190475,
 0.20809248554913284,
 0.17703349282296665,
 0.0934959349593496,
 -0.11895910780669139,
 -0.2616033755274262,
 0.005714285714285783,
 0.02840909090909083,
 0.1049723756906078,
 0.20999999999999996,
 -0.004132231404958553,
 0.13692946058091282,
 -0.051094890510948954,
 -0.011538461538461608,
 -0.0038910505836574627,
 -0.10156250000000011,
 0.06956521739130439,
 0.012195121951219523,
 -0.048192771084337394,
 -0.0042194092827004814,
 0.025423728813559254,
 -0.004132231404958553,
 0.024896265560165887,
 0.05263157894736836,
 0.05384615384615388,
 -0.010948905109489093,
 -0.1254612546125461,
 -0.10548523206751059,
 -0.05660377358490576,
 -0.12,
 -0.22727272727272718,
 -0.13235294117647067,
 0.10169491525423746,
 0.10769230769230753,
 -0.08333333333333326,
 0.045454545454545414,
 0.1304347826086958,
 0.07051282051282048,
 0.041916167664670656,
 -0.12643678160919536,
 -0.07894736842105265,
 -0.2142857142857142,
 -0.7272727272727273,
 0.2333333333333334,
 -1.4054054054054053,
 1.7333333333333334,
 0.4878048780487805,
 -0.049180327868852514,
 0.2586206896551724,
 -0.1095890410958904,
 0.1384615384615384,
 -0.33783783783783783,
 0.30612244897959195,
 0.34375,
 0.5697674418604652,
 0.42222222222222205,
 -0.13020833333333337,
 0.317365269461078,
 -0.20454545454545459,
 0.8742857142857141,
 -0.27439024390243905,
 -1.5420168067226891,
 0.9224806201550386,
 -0.22983870967741937,
 -0.15183246073298418,
 -0.49382716049382724,
 -1.475609756097561,
 3.4615384615384617,
 1.0172413793103448,
 -0.07122507122507127,
 -0.2668711656441717,
 -0.6234309623430963,
 -0.22222222222222232,
 3.4857142857142858,
 -0.35031847133757965,
 -0.27941176470588236,
 -0.027210884353741527,
 -0.8881118881118881,
 -6.874999999999999,
 1.2021276595744679,
 0.16425120772946866,
 -0.27800829875518673,
 -0.8390804597701149,
 1.0714285714285712,
 -0.017241379310344862,
 0.7192982456140351,
 0.2755102040816326,
 0.6800000000000002,
 0.9333333333333331,
 0.022167487684729315,
 -0.2915662650602411,
 -0.2517006802721088,
 0.05454545454545445,
 0.004310344827586299,
 0.0,
 -0.20171673819742486,
 0.021505376344086002,
 0.0,
 -0.0842105263157894,
 0.12643678160919536,
 0.09693877551020402,
 0.07441860465116279,
 0.11688311688311681,
 0.011627906976744207,
 -0.045977011494252706,
 -0.0200803212851407,
 -0.004098360655737654,
 0.05349794238683114,
 0.24609375,
 0.034482758620689724,
 -0.14242424242424234,
 -0.3568904593639576,
 -0.05494505494505497,
 0.18604651162790709,
 0.2990196078431371,
 0.19245283018867942,
 0.06012658227848089,
 -0.11940298507462688,
 0.027118644067796405,
 0.05610561056105623,
 -0.025000000000000022,
 -0.04166666666666663,
 0.0033444816053511683,
 -0.013333333333333308,
 0.027027027027026973,
 -0.0625,
 -0.19298245614035092,
 -0.16086956521739126,
 -0.020725388601036343,
 -0.25396825396825395,
 -0.276595744680851,
 0.03921568627450989,
 0.3018867924528301,
 0.01449275362318847,
 -0.07142857142857129,
 0.1384615384615384,
 0.4189189189189191,
 0.004761904761904745,
 -0.17061611374407581,
 -0.1885714285714286,
 0.05633802816901423,
 0.0,
 0.0,
 0.4800000000000002,
 0.2567567567567566,
 -0.07885304659498216,
 0.011673151750972943,
 -0.023076923076923106,
 0.11023622047244097,
 0.13829787234042556,
 -0.06230529595015577,
 0.013289036544850585,
 -0.11803278688524588,
 -0.06691449814126404,
 0.05179282868525914,
 0.0,
 -0.14393939393939403,
 -0.06194690265486713,
 -0.07075471698113212,
 -0.23857868020304562,
 -0.19333333333333336,
 -0.23140495867768585,
 -0.18279569892473124,
 -0.30263157894736836,
 -0.37735849056603776,
 -0.36363636363636365,
 0.0,
 -0.38095238095238093,
 -2.230769230769231,
 -0.1875,
 -0.5384615384615385,
 -4.0,
 -0.38888888888888884,
 -1.7272727272727273,
 -0.375,
 -7.199999999999999,
 0.5161290322580645,
 -0.10638297872340419,
 0.7857142857142858,
 0.01333333333333342,
 -0.35526315789473684,
 0.20408163265306123,
 0.7796610169491527,
 0.22857142857142865,
 0.023255813953488413,
 -0.16666666666666663,
 0.027272727272727115,
 0.48672566371681425,
 0.03571428571428581,
 0.16666666666666652,
 0.09852216748768483,
 0.10313901345291487,
 0.03658536585365857,
 -0.007843137254901933,
 -0.04347826086956519,
 -0.05371900826446274,
 0.05240174672489095,
 0.13692946058091282,
 0.06569343065693412,
 0.07534246575342474,
 0.08280254777070062,
 0.07058823529411762,
 0.01098901098901095,
 -0.04619565217391319,
 0.014245014245014342,
 -0.047752808988763995,
 0.017699115044247815,
 0.06376811594202891,
 0.0,
 -0.05177111716621252,
 0.01724137931034475,
 -0.07627118644067798,
 -0.09174311926605494,
 0.023569023569023573,
 -0.006578947368421018,
 -0.06622516556291391,
 -0.04255319148936154,
 -0.040740740740740855,
 -0.09266409266409259,
 -0.04680851063829783,
 0.13839285714285698,
 0.03529411764705892,
 0.02651515151515138,
 -0.022140221402214055,
 0.09433962264150941,
 0.10344827586206895,
 -0.08125000000000004,
 -0.027210884353741527,
 0.0,
 -0.07342657342657344,
 0.026415094339622858,
 -0.022058823529411797,
 -0.052631578947368474,
 -0.18650793650793662,
 -0.06829268292682922,
 -0.19895287958115182,
 -0.14379084967320255,
 -0.04580152671755733,
 -0.36,
 -0.3125,
 0.2909090909090908,
 0.323943661971831,
 -0.18085106382978722,
 -0.20779220779220786,
 -0.3114754098360656,
 0.023809523809523725,
 0.2093023255813955,
 1.2692307692307692,
 0.21186440677966112,
 0.104895104895105,
 0.06329113924050622,
 -0.05952380952380942,
 -0.08227848101265833,
 0.10344827586206895,
 -0.11875000000000013,
 -0.2695035460992907,
 0.2233009708737863,
 0.11904761904761907,
 -0.0921985815602836,
 0.1015625,
 0.1276595744680853,
 -0.04402515723270439,
 -0.05921052631578949,
 -0.2797202797202797,
 -0.009708737864077666,
 0.1078431372549018,
 -0.1769911504424777,
 -0.3548387096774194,
 -0.1499999999999999,
 -0.2941176470588236,
 -0.05555555555555547,
 0.4117647058823528,
 0.16666666666666674,
 -0.07142857142857151,
 -0.25,
 -0.05128205128205132,
 -0.16216216216216217,
 -0.7419354838709677,
 4.874999999999999,
 -0.34042553191489355,
 -0.5161290322580645,
 0.8000000000000003,
 0.6666666666666665,
 0.5111111111111111,
 0.1470588235294117,
 0.1923076923076923,
 0.30107526881720426,
 -0.07438016528925606,
 -0.017857142857142905,
 0.0181818181818183,
 0.0,
 -0.267857142857143,
 0.14634146341463405,
 0.26595744680851063,
 -0.6554621848739496,
 -0.5853658536585366,
 1.8235294117647056,
 -0.47916666666666663,
 -1.3599999999999999,
 3.8888888888888893,
 -0.13636363636363635,
 0.44736842105263164,
 0.14545454545454528,
 0.11111111111111094,
 -0.8142857142857143,
 -1.6923076923076923,
 3.0,
 2.2777777777777777,
 0.43220338983050843,
 0.01183431952662728,
 -0.03508771929824561,
 -0.07272727272727264,
 0.33333333333333326,
 0.16176470588235303,
 0.16033755274261607,
 0.22545454545454557,
 -0.0029673590504452063,
 -0.05952380952380942,
 0.09493670886075933,
 0.0028901734104047616,
 -0.02017291066282434,
 -0.05882352941176461,
 -0.08125000000000004,
 -0.11224489795918369,
 -0.15325670498084287,
 0.05429864253393668,
 0.20171673819742475,
 0.0071428571428571175,
 0.014184397163120588,
 -0.05244755244755239,
 -0.01845018450184499,
 0.05639097744360888,
 -0.11743772241992889,
 -0.032258064516129115,
 0.27916666666666656,
 0.1335504885993486,
 -0.048850574712643646,
 0.015105740181268867,
 -0.0029761904761904656,
 0.0029850746268655914,
 -0.029761904761904767,
 -0.11963190184049066,
 0.1811846689895471,
 0.08554572271386429,
 -0.0652173913043479,
 -0.08430232558139539,
 -0.12063492063492065,
 -0.1155234657039711,
 -0.05714285714285716,
 -0.09956709956709953,
 -0.03846153846153855,
 -0.07999999999999996,
 -0.14130434782608692,
 0.07594936708860756,
 -0.11764705882352944,
 -0.17333333333333334,
 -0.217741935483871,
 -0.08247422680412364,
 -0.1685393258426966,
 -0.05405405405405406,
 -0.05714285714285705,
 -0.13636363636363646,
 -0.14035087719298245,
 -0.8571428571428571,
 -0.5714285714285715,
 2.0,
 2.0000000000000004,
 0.0,
 -0.2962962962962963,
 -0.9473684210526316,
 -22.0,
 0.04761904761904767,
 0.4545454545454546,
 0.4687499999999998,
 -0.10638297872340419,
 -0.16666666666666674,
 0.2857142857142858,
 0.1333333333333333,
 -0.3921568627450981,
 -0.6774193548387096,
 -4.699999999999999,
 -0.8648648648648649,
 6.3999999999999995,
 0.45945945945945965,
 0.0,
 0.5,
 0.28395061728395055,
 -0.10576923076923073,
 1.397849462365591,
 0.06278026905829592,
 -0.10548523206751059,
 0.04245283018867907,
 0.06334841628959276,
 -0.08936170212765959,
 0.1869158878504673,
 0.23228346456692917,
 0.06709265175718859,
 -0.28443113772455086,
 0.0,
 0.07531380753138062,
 0.015564202334630295,
 0.06130268199233724,
 0.12274368231046928,
 0.13826366559485526,
 -0.04519774011299438,
 0.01183431952662728,
 -0.040935672514619936,
 0.012195121951219523,
 0.009036144578313365,
 0.05671641791044779,
 0.03672316384180796,
 -0.024523160762942697,
 0.0,
 0.030726256983240274,
 -0.1165311653116532,
 -0.05521472392638027,
 -0.07467532467532467,
 -0.10877192982456141,
 -0.015748031496062964,
 -0.03599999999999992,
 0.08713692946058083,
 0.2022900763358777,
 0.028571428571428692,
 0.06481481481481488,
 -0.04057971014492756,
 0.027190332326283873,
 -0.0794117647058824,
 -0.054313099041533475,
 0.0,
 -0.22972972972972983,
 -0.13596491228070173,
 0.08121827411167515,
 -0.06103286384976525,
 -0.015000000000000013,
 -0.015228426395939132,
 0.07731958762886593,
 -0.0574162679425837,
 -0.13197969543147203,
 -0.10526315789473684,
 -0.06535947712418311,
 0.104895104895105,
 0.018987341772152,
 0.024844720496894235,
 -0.05454545454545445,
 0.05769230769230771,
 0.11515151515151523,
 0.021739130434782483,
 -0.005319148936170137,
 -0.09090909090909094,
 0.11176470588235299,
 0.19047619047619047,
 0.12888888888888883,
 0.06299212598425208,
 0.033333333333333215,
 -0.07885304659498216,
 0.031128404669260812,
 0.06792452830188678,
 -0.0035335689045937757,
 -0.05673758865248213,
 0.003759398496240518,
 0.0037453183520599342,
 -0.05597014925373145,
 0.011857707509881577,
 -0.01953125000000011,
 -0.04780876494023889,
 0.05020920502092041,
 -0.0916334661354582,
 0.013157894736842257,
 -0.05627705627705626,
 -0.1513761467889908,
 0.059459459459459296,
 0.025510204081632626,
 -0.04477611940298498,
 0.13541666666666674,
 0.07339449541284382,
 -0.02136752136752129,
 -0.08296943231441045,
 0.023809523809523725,
 -0.04651162790697683,
 0.043902439024390505,
 -0.06074766355140204,
 -0.08955223880596996,
 -0.12568306010928965,
 -0.012499999999999956,
 -0.025316455696202556,
 -0.11038961038961037,
 -0.12408759124087598,
 0.050000000000000044,
 0.06349206349206349,
 0.06716417910447747,
 0.1748251748251748,
 0.17261904761904767,
 -0.030456852791878153,
 -0.010471204188481686,
 -0.08465608465608465,
 -0.138728323699422,
 -0.06040268456375841,
 -0.15000000000000002,
 0.03361344537815136,
 -0.04065040650406504,
 -0.025423728813559365,
 0.10434782608695659,
 -0.13385826771653542,
 -0.036363636363636376,
 0.08490566037735836,
 0.09565217391304359,
 -0.11904761904761896,
 -0.027027027027027084,
 0.0092592592592593,
 -0.11009174311926617,
 -0.08247422680412364,
 -0.07865168539325851,
 0.012195121951219523,
 0.03614457831325302,
 -0.13953488372093026,
 -0.44594594594594594,
 -0.29268292682926833,
 -0.1724137931034483,
 -0.5,
 -0.16666666666666663,
 -1.0,
 0.0,
 -0.4285714285714286,
 3.5,
 -0.36111111111111105,
 -1.1304347826086956,
 7.0,
 0.20833333333333326,
 -0.27586206896551724,
 1.7619047619047619,
 -0.10344827586206884,
 0.03846153846153855,
 0.05555555555555536,
 -0.14035087719298245,
 0.12244897959183687,
 0.036363636363636154,
 0.21052631578947367,
 0.1304347826086958,
 0.07692307692307687,
 0.19047619047619047,
 0.21999999999999997,
 0.29508196721311486,
 0.025316455696202445,
 -0.012345679012345734,
 -0.07500000000000007,
 -0.14189189189189189,
 -0.03149606299212604,
 0.08130081300813008,
 0.15037593984962405,s
 -0.013071895424836666,
 -0.06622516556291391,
 0.14184397163120588,
 -0.006211180124223614,
 0.04999999999999982,
 0.1785714285714286,
 -0.04040404040404044,
 -0.14736842105263148,
 -0.6111111111111112,
 -0.6507936507936508,
 0.4545454545454546,
 -0.46875,
 -3.117647058823529,
 1.0555555555555558,
 0.527027027027027])

In [103]:
def count_momentum(change_list):
    momentum_list = [0] * len(change_list)

    for index in change_list.index:
        count = index
        if pd.isnull(change_list[index]):  
            momentum_list[index] = None

        if change_list[index] > 0:
                while change_list[count] >= 0:
                    momentum_list[index] += 1
                    count -= 1

        if change_list[index] < 0:
                while change_list[count] <= 0:
                    momentum_list[index] -= 1
                    count -= 1
    return momentum_list

In [104]:
count_momentum(test)

[None,
 1,
 -1,
 -2,
 1,
 2,
 3,
 -1,
 1,
 -1,
 1,
 2,
 -1,
 -2,
 1,
 -1,
 -2,
 -3,
 0,
 2,
 -1,
 -2,
 1,
 -1,
 1,
 -1,
 -2,
 -3,
 -4,
 -5,
 1,
 0,
 -2,
 -3,
 1,
 2,
 -1,
 -2,
 -3,
 1,
 2,
 3,
 -1,
 1,
 2,
 3,
 4,
 -1,
 1,
 2,
 -1,
 -2,
 1,
 -1,
 -2,
 1,
 -1,
 1,
 2,
 3,
 4,
 -1,
 1,
 -1,
 -2,
 -3,
 1,
 -1,
 -2,
 -3,
 -4,
 -5,
 -6,
 -7,
 -8,
 1,
 -1,
 1,
 2,
 -1,
 -2,
 1,
 -1,
 1,
 -1,
 1,
 2,
 -1,
 -2,
 -3,
 -4,
 -5,
 -6,
 -7,
 -8,
 1,
 2,
 3,
 4,
 5,
 -1,
 -2,
 1,
 2,
 3,
 4,
 -1,
 1,
 -1,
 -2,
 -3,
 -4,
 1,
 2,
 -1,
 -2,
 1,
 -1,
 1,
 2,
 3,
 -1,
 -2,
 -3,
 -4,
 -5,
 -6,
 -7,
 1,
 2,
 -1,
 1,
 2,
 3,
 4,
 -1,
 -2,
 -3,
 -4,
 1,
 -1,
 1,
 2,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 2,
 3,
 4,
 -1,
 1,
 -1,
 1,
 -1,
 -2,
 1,
 -1,
 -2,
 -3,
 -4,
 1,
 2,
 -1,
 -2,
 -3,
 -4,
 1,
 -1,
 -2,
 -3,
 -4,
 -5,
 1,
 2,
 -1,
 -2,
 1,
 -1,
 1,
 2,
 3,
 4,
 5,
 -1,
 -2,
 1,
 2,
 0,
 -2,
 1,
 0,
 -2,
 1,
 2,
 3,
 4,
 5,
 -1,
 -2,
 -3,
 1,
 2,
 3,
 -1,
 -2,
 -3,
 1,
 2,
 3,
 4,
 -1,
 1,
 2,
 -1,
 -2,
 1,
 -1,
 1,